In [1]:
# Following exactly the Germans' code -- found their mistake!!!

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats
from numpy.linalg import inv
import pandas_datareader as pdr

In [3]:
# Load required data
ff_reversal = pdr.get_data_famafrench("F-F_ST_Reversal_Factor", start="1963-07", end="2020-06")[0] * (1/100)
ff_momentum = pdr.get_data_famafrench("F-F_Momentum_Factor", start="1963-07", end="2020-06")[0] * (1/100)
ff_factors = pdr.get_data_famafrench("F-F_Research_Data_Factors", start="1963-07", end="2020-06")[0] * (1/100)
ff_5_factors = pdr.get_data_famafrench("F-F_Research_Data_5_Factors_2x3", start="1963-07", end="2020-06")[0] * (1/100)
ff_portfolios = pdr.get_data_famafrench("25_Portfolios_5x5", start="1963-07", end="2020-06")[0] * (1/100)

# Portfolio returns must be excess for GRS
ff_portfolios_excess = ff_portfolios.sub(ff_factors["RF"], axis=0)

# Remove RF from dataframes
ff_5_factors = ff_5_factors.drop(columns=["RF"])
ff_factors = ff_factors.drop(columns=["RF"])

#For 5 Factors
ff_5_factors_constant = sm.add_constant(ff_5_factors)

# For 3 Factors + Momentum
ff_3_factors_mom = pd.concat([ff_factors["Mkt-RF"], ff_factors["SMB"], ff_factors["HML"], ff_momentum], axis=1)
ff_3_factors_mom_constant = sm.add_constant(ff_3_factors_mom)

# For 3 Factors + reversal
ff_3_factors_rev = pd.concat([ff_factors["Mkt-RF"], ff_factors["SMB"], ff_factors["HML"], ff_reversal], axis=1)
ff_3_factors_rev_constant = sm.add_constant(ff_3_factors_rev)

In [4]:
ff_factors

,Mkt-RF,SMB,HML
Date,,,
1963-07,-0.0039,-0.0049,-0.0094
1963-08,0.0507,-0.0102,0.0182
1963-09,-0.0157,-0.0031,0.0017
1963-10,0.0253,-0.0057,-0.0004
1963-11,-0.0085,-0.0115,0.0170
...,...,...,...
2020-02,-0.0813,0.0102,-0.0392
2020-03,-0.1338,-0.0503,-0.1396
2020-04,0.1365,0.0275,-0.0139


In [5]:
def calc_GRS(alpha_vector, factor_dataframe, asset_returns):
    gen_cov = factor_dataframe.cov()
    generalized_sharpe = factor_dataframe.mean().T @ inv(gen_cov) @ factor_dataframe.mean()     #Cochrane 2005, p.217
    cov = asset_returns.cov()
    if isinstance(alpha_vector, list):
        alpha_vector = np.array(alpha_vector)
    #Calc GRS
    df = len(asset_returns) - len(asset_returns.columns) - len(factor_dataframe.columns) # T - N - K     (N = number of assets, K = number of factors)
    N = len(asset_returns.columns)
    GRS = (df/ N) * ((alpha_vector.T @ inv(cov) @ alpha_vector) / (1+ generalized_sharpe))
    p = 1-scipy.stats.f.cdf(GRS, len(asset_returns.columns), df) #find p-value of F test statistic
    return GRS, p

In [6]:
alphas_3 = []

alphas_5 = []
alphas_mom = []
alphas_rev = []

for columns in ff_portfolios_excess:

    x = sm.add_constant(ff_factors)
    res = sm.OLS(ff_portfolios_excess[columns],x).fit()
    alphas_3.append(res.params[0])

    x = sm.add_constant(ff_5_factors)
    res = sm.OLS(ff_portfolios_excess[columns],x).fit()
    alphas_5.append(res.params[0])

    x = sm.add_constant(ff_3_factors_mom)
    res = sm.OLS(ff_portfolios_excess[columns],x).fit()
    alphas_mom.append(res.params[0])

    x = sm.add_constant(ff_3_factors_rev)
    res = sm.OLS(ff_portfolios_excess[columns],x).fit()
    alphas_rev.append(res.params[0])


In [7]:
## NOT NEEDED HERE: for comparison with 3d - 3 factor non-investment portfolio.
calc_GRS(alphas_3,ff_factors,ff_portfolios)

(3.4774738491444026, 3.765973921598942e-08)

In [8]:
calc_GRS(alphas_5,ff_5_factors,ff_portfolios)

(2.8519056165283048, 5.697332044030112e-06)

In [9]:
calc_GRS(alphas_mom,ff_3_factors_mom,ff_portfolios)

(2.7369360415319433, 1.3843179106909353e-05)

In [10]:
calc_GRS(alphas_rev,ff_3_factors_rev,ff_portfolios)

(3.605381005273485, 1.3149262900746805e-08)

In [11]:
## 5 FACTOR MODEL

B_mkt = []
B_SMB = []
B_HML = []
B_RMW = []
B_CMA = []

#Step one:
for column in ff_portfolios_excess:
    x = ff_5_factors_constant
    y = ff_portfolios_excess[column]
    reg = sm.OLS(y,x).fit()
    B_mkt.append(reg.params[1])
    B_SMB.append(reg.params[2])
    B_HML.append(reg.params[3])
    B_RMW.append(reg.params[4])
    B_CMA.append(reg.params[5])

B_mkt = pd.DataFrame(np.array(B_mkt))
B_SMB = pd.DataFrame(np.array(B_SMB))
B_HML = pd.DataFrame(np.array(B_HML))
B_RMW = pd.DataFrame(np.array(B_RMW))
B_CMA = pd.DataFrame(np.array(B_CMA))

betas = pd.concat([B_mkt, B_SMB, B_HML, B_RMW, B_CMA], axis=1)
betas.columns = ['B_mkt', "B_SMB", 'B_HML', "B_RMW", 'B_CMA']
betas = sm.add_constant(betas)
betas.index = ff_portfolios_excess.T.index

alpha = []
lambda_mkt = []
lambda_SMB = []
lambda_HML = []
lambda_RMW = []
lambda_CMA = []

for column in ff_portfolios_excess.T:
    x = betas
    y = ff_portfolios_excess.T[column]
    reg = sm.OLS(y,x).fit()
    alpha.append(reg.params[0])
    lambda_mkt.append(reg.params[1])
    lambda_SMB.append(reg.params[2])
    lambda_HML.append(reg.params[3])
    lambda_RMW.append(reg.params[4])
    lambda_CMA.append(reg.params[5]) #appends the alphas and risk premia for every time period

alpha = pd.DataFrame(np.array(alpha))
lambda_mkt = pd.DataFrame(np.array(lambda_mkt))
lambda_SMB = pd.DataFrame(np.array(lambda_SMB))
lambda_HML = pd.DataFrame(np.array(lambda_HML))
lambda_RMW = pd.DataFrame(np.array(lambda_RMW))
lambda_CMA = pd.DataFrame(np.array(lambda_CMA))

lambdas = pd.concat([lambda_mkt, lambda_SMB, lambda_HML, lambda_RMW, lambda_CMA], axis=1)
lambdas.columns = ['lambda_mkt', "lambda_SMB", 'lambda_HML', "lambda_RMW", 'lambda_CMA']

params = [alpha, lambda_mkt, lambda_SMB, lambda_HML, lambda_RMW, lambda_CMA]

variances = []

for i in params:
    var = ((i.sub(i.mean(), axis=1)) ** 2).sum() / (len(i) ** 2)
    variances.append(var)

variances = pd.DataFrame(np.array(variances)).T
variances.columns = ['alpha', 'lambda_mkt', 'lambda_SMB', 'lambda_HML', 'lambda_RMW', 'lambda_CMA']

#Shanken Adjustment
adjustment = (1 + (lambdas.mean().T @ inv(ff_5_factors.cov()) @ lambdas.mean()))            # DO we not need the applicative adjustment too (check with prof, Cochrane 2005 p.223)
#Adjusted variances
variances = variances * adjustment

#t-tests
t_alpha = alpha.mean() / np.sqrt(variances['alpha'])
t_mkt = lambda_mkt.mean() / np.sqrt(variances['lambda_mkt'])
t_SMB = lambda_SMB.mean() / np.sqrt(variances['lambda_SMB'])
t_HML = lambda_HML.mean() / np.sqrt(variances['lambda_HML'])
t_RMW = lambda_RMW.mean() / np.sqrt(variances['lambda_RMW'])
t_CMA = lambda_CMA.mean() / np.sqrt(variances['lambda_CMA'])
t_stats = pd.concat([t_alpha, t_mkt, t_SMB, t_HML, t_RMW, t_CMA], axis=1)
t_stats.columns = ['t_alpha', 't_mkt', 't_SMB', 't_HML', 't_RMW', 't_CMA']
t_stats = np.round(t_stats,2)

print("Factor premia and pricing error for the 5 factor model are:")
print("Pricing error:", alpha.mean(), "Premia:", np.round(lambdas.mean(),4))
print("t-stats:", t_stats)

Factor premia and pricing error for the 5 factor model are:
Pricing error: 0    0.009395
dtype: float64 Premia: lambda_mkt   -0.0042
lambda_SMB    0.0026
lambda_HML    0.0022
lambda_RMW    0.0044
lambda_CMA    0.0002
dtype: float64
t-stats:    t_alpha  t_mkt  t_SMB  t_HML  t_RMW  t_CMA
0     3.38  -1.28   2.15    1.9   2.54   0.11


In [12]:
## 3 FACTOR MODEL + momentum

B_mkt = []
B_SMB = []
B_HML = []
B_MOM = []

#Step one:
for column in ff_portfolios_excess:
    x = ff_3_factors_mom_constant
    y = ff_portfolios_excess[column]
    reg = sm.OLS(y,x).fit()
    B_mkt.append(reg.params[1])
    B_SMB.append(reg.params[2])
    B_HML.append(reg.params[3])
    B_MOM.append(reg.params[4])
  

B_mkt = pd.DataFrame(np.array(B_mkt))
B_SMB = pd.DataFrame(np.array(B_SMB))
B_HML = pd.DataFrame(np.array(B_HML))
B_MOM = pd.DataFrame(np.array(B_MOM))


betas = pd.concat([B_mkt, B_SMB, B_HML, B_MOM], axis=1)
betas.columns = ['B_mkt', "B_SMB", 'B_HML', 'B_MOM']
betas = sm.add_constant(betas)
betas.index = ff_portfolios_excess.T.index

alpha = []
lambda_mkt = []
lambda_SMB = []
lambda_HML = []
lambda_MOM = []

for column in ff_portfolios_excess.T:
    x = betas
    y = ff_portfolios_excess.T[column]
    reg = sm.OLS(y,x).fit()
    alpha.append(reg.params[0])
    lambda_mkt.append(reg.params[1])
    lambda_SMB.append(reg.params[2])
    lambda_HML.append(reg.params[3])
    lambda_MOM.append(reg.params[4])  #appends the alphas and risk premia for every time period

alpha = pd.DataFrame(np.array(alpha))
lambda_mkt = pd.DataFrame(np.array(lambda_mkt))
lambda_SMB = pd.DataFrame(np.array(lambda_SMB))
lambda_HML = pd.DataFrame(np.array(lambda_HML))
lambda_MOM = pd.DataFrame(np.array(lambda_MOM))

lambdas = pd.concat([lambda_mkt, lambda_SMB, lambda_HML, lambda_MOM], axis=1)
lambdas.columns = ['lambda_mkt', "lambda_SMB", 'lambda_HML', "lambda_MOM"]

params = [alpha, lambda_mkt, lambda_SMB, lambda_HML, lambda_MOM]

variances = []

for i in params:
    var = ((i.sub(i.mean(), axis=1)) ** 2).sum() / (len(i) ** 2)
    variances.append(var)

variances = pd.DataFrame(np.array(variances)).T
variances.columns = ['alpha', 'lambda_mkt', 'lambda_SMB', 'lambda_HML', 'lambda_MOM']

#Shanken Adjustment
adjustment = (1 + (lambdas.mean().T @ inv(ff_3_factors_mom.cov()) @ lambdas.mean()))            # DO we not need the applicative adjustment too (check with prof, Cochrane 2005 p.223)
#Adjusted variances
variances = variances * adjustment

#t-tests
t_alpha = alpha.mean() / np.sqrt(variances['alpha'])
t_mkt = lambda_mkt.mean() / np.sqrt(variances['lambda_mkt'])
t_SMB = lambda_SMB.mean() / np.sqrt(variances['lambda_SMB'])
t_HML = lambda_HML.mean() / np.sqrt(variances['lambda_HML'])
t_MOM = lambda_MOM.mean() / np.sqrt(variances['lambda_MOM'])
t_stats = pd.concat([t_alpha, t_mkt, t_SMB, t_HML, t_MOM], axis=1)
t_stats.columns = ['t_alpha', 't_mkt', 't_SMB', 't_HML', 't_MOM']
t_stats = np.round(t_stats,2)

print("Factor premia and pricing error for the 3 and mom factor model are:")
print("Pricing error:", alpha.mean(), "Premia:", np.round(lambdas.mean(),4))
print("t-stats:", t_stats)

Factor premia and pricing error for the 3 and mom factor model are:
Pricing error: 0    0.004039
dtype: float64 Premia: lambda_mkt    0.0020
lambda_SMB    0.0015
lambda_HML    0.0031
lambda_MOM    0.0249
dtype: float64
t-stats:    t_alpha  t_mkt  t_SMB  t_HML  t_MOM
0     1.11   0.47   1.01   2.29   2.98


In [13]:
reg.params

const    0.119107
B_mkt   -0.111711
B_SMB    0.036199
B_HML   -0.023165
B_MOM   -0.400634
dtype: float64

In [14]:
## 3 FACTOR MODEL + reversal

B_mkt = []
B_SMB = []
B_HML = []
B_REV = []

#Step one:
for column in ff_portfolios_excess:
    x = ff_3_factors_rev_constant
    y = ff_portfolios_excess[column]
    reg = sm.OLS(y,x).fit()
    B_mkt.append(reg.params[1])
    B_SMB.append(reg.params[2])
    B_HML.append(reg.params[3])
    B_REV.append(reg.params[4])
  

B_mkt = pd.DataFrame(np.array(B_mkt))
B_SMB = pd.DataFrame(np.array(B_SMB))
B_HML = pd.DataFrame(np.array(B_HML))
B_REV = pd.DataFrame(np.array(B_REV))


betas = pd.concat([B_mkt, B_SMB, B_HML, B_REV], axis=1)
betas.columns = ['B_mkt', "B_SMB", 'B_HML', 'B_REV']
betas = sm.add_constant(betas)
betas.index = ff_portfolios_excess.T.index

alpha = []
lambda_mkt = []
lambda_SMB = []
lambda_HML = []
lambda_REV = []

for column in ff_portfolios_excess.T:
    x = betas
    y = ff_portfolios_excess.T[column]
    reg = sm.OLS(y,x).fit()
    alpha.append(reg.params[0])
    lambda_mkt.append(reg.params[1])
    lambda_SMB.append(reg.params[2])
    lambda_HML.append(reg.params[3])
    lambda_REV.append(reg.params[4])  #appends the alphas and risk premia for every time period

alpha = pd.DataFrame(np.array(alpha))
lambda_mkt = pd.DataFrame(np.array(lambda_mkt))
lambda_SMB = pd.DataFrame(np.array(lambda_SMB))
lambda_HML = pd.DataFrame(np.array(lambda_HML))
lambda_REV = pd.DataFrame(np.array(lambda_REV))

lambdas = pd.concat([lambda_mkt, lambda_SMB, lambda_HML, lambda_REV], axis=1)
lambdas.columns = ['lambda_mkt', "lambda_SMB", 'lambda_HML', "lambda_REV"]

params = [alpha, lambda_mkt, lambda_SMB, lambda_HML, lambda_REV]

variances = []

for i in params:
    var = ((i.sub(i.mean(), axis=1)) ** 2).sum() / (len(i) ** 2)
    variances.append(var)

variances = pd.DataFrame(np.array(variances)).T
variances.columns = ['alpha', 'lambda_mkt', 'lambda_SMB', 'lambda_HML', 'lambda_REV']

#Shanken Adjustment
adjustment = (1 + (lambdas.mean().T @ inv(ff_3_factors_rev.cov()) @ lambdas.mean()))            # DO we not need the applicative adjustment too (check with prof, Cochrane 2005 p.223)
#Adjusted variances
variances = variances * adjustment

#t-tests
t_alpha = alpha.mean() / np.sqrt(variances['alpha'])
t_mkt = lambda_mkt.mean() / np.sqrt(variances['lambda_mkt'])
t_SMB = lambda_SMB.mean() / np.sqrt(variances['lambda_SMB'])
t_HML = lambda_HML.mean() / np.sqrt(variances['lambda_HML'])
t_REV = lambda_REV.mean() / np.sqrt(variances['lambda_REV'])
t_stats = pd.concat([t_alpha, t_mkt, t_SMB, t_HML, t_REV], axis=1)
t_stats.columns = ['t_alpha', 't_mkt', 't_SMB', 't_HML', 't_REV']
t_stats = np.round(t_stats,2)

print("Factor premia and pricing error for the 3 and rev factor model are:")
print("Pricing error:", alpha.mean(), "Premia:", np.round(lambdas.mean(),4))
print("t-stats:", t_stats)

Factor premia and pricing error for the 3 and rev factor model are:
Pricing error: 0    0.011386
dtype: float64 Premia: lambda_mkt   -0.0057
lambda_SMB    0.0014
lambda_HML    0.0029
lambda_REV   -0.0071
dtype: float64
t-stats:    t_alpha  t_mkt  t_SMB  t_HML  t_REV
0     4.25  -1.77   1.16   2.45  -1.53
